<a href="https://colab.research.google.com/github/poorya99/keras_deeplearning/blob/main/fft_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Runtime > Change runtime type > Setting the Hardware accelerator to GPU > Save

https://github.com/flin3500/cuda-google-colab

https://harshityadav95.medium.com/how-to-run-cuda-c-or-c-on-google-colab-or-azure-notebook-ea75a23a5962

https://stackoverflow.com/questions/72104648/how-can-i-fix-this-dpkg-error-while-installing-cuda-on-google-colab

In [24]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [25]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-6g9hxuqa
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-6g9hxuqa


In [26]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [27]:
%%writefile fft.cu
//ONLY MODIFY THIS FILE!
//YOU CAN MODIFY EVERYTHING IN THIS FILE!

#include "fft.h"
#include <stdint.h>
#include <cstdint>

#define tx threadIdx.x
#define ty threadIdx.y
#define tz threadIdx.z

#define bx blockIdx.x
#define by blockIdx.y
#define bz blockIdx.z

// you may define other parameters here!
__device__ int reverse_bits_4(int x)
{
    
    // 1. Swap the position of consecutive pairs of bits
    // 2. Swap the position of consecutive quads of bits
    // 3. Continue this until swapping the two consecutive 16-bit parts of x
    
    x = ((x & 0xcccccccc) >> 2) | ((x & 0x33333333) << 2);
    x = ((x & 0xf0f0f0f0) >> 4) | ((x & 0x0f0f0f0f) << 4);
    x = ((x & 0xff00ff00) >> 8) | ((x & 0x00ff00ff) << 8);
    x = ((x & 0xffff0000) >> 16) | ((x & 0x0000ffff) << 16);
    return x;
}


__global__ void kernelFunc(float* x_r, float* x_i, const int N, const int M) {

	// write your GPU kernel function here

    //int row = by * blockDim.y + ty;
    //int col = bx * blockDim.x + tx;
  
    int logN = (int) log2f((float) N);

	int Idx = bx * blockDim.x + tx;
 
  
	// Reverse the 32-bit index.
	unsigned int rev0 = reverse_bits_4(4*Idx);
	unsigned int rev1 = reverse_bits_4(4*Idx+1);
	unsigned int rev2 = reverse_bits_4(4*Idx+2);
	unsigned int rev3 = reverse_bits_4(4*Idx+3);
	// Only keep the last logN bits of the output.
	rev0 = rev0 >> (32 - logN);
	rev1 = rev1 >> (32 - logN);
	rev2 = rev2 >> (32 - logN);
	rev3 = rev3 >> (32 - logN);
  
  //printf("\n Idx = %d(%d, %d, %d, %d) ", Idx, rev0,rev1,rev2,rev3);

	// Base case: set the output to the bit-reversed input.
	float x_tmp_r0 = x_r[rev0];
	float x_tmp_i0 = x_i[rev0];
	float x_tmp_r1 = x_r[rev1];
	float x_tmp_i1 = x_i[rev1];
	
	float x_tmp_r2 = x_r[rev2];
	float x_tmp_i2 = x_i[rev2];
	float x_tmp_r3 = x_r[rev3];
	float x_tmp_i3 = x_i[rev3];
	
    __syncthreads();
  x_r[4*Idx  ] = x_tmp_r0;
	x_i[4*Idx  ] = x_tmp_i0;
	x_r[4*Idx+1] = x_tmp_r1;
	x_i[4*Idx+1] = x_tmp_i1;
	x_r[4*Idx+2] = x_tmp_r2;
	x_i[4*Idx+2] = x_tmp_i2;
	x_r[4*Idx+3] = x_tmp_r3;
	x_i[4*Idx+3] = x_tmp_i3;

	__syncthreads();
  //**************************************************************
if(Idx == 0){
  int d = 0;
  int xx = ~d;

  printf("xxxxxxx%u, ", sizeof(xx));


for(uint32_t deb1 = 0;deb1<N;deb1++){
  uint32_t intt = ((reverse_bits_4(deb1)));
  intt = (unsigned int)intt>> (32 - logN);
  //intt = intt & 0x‭7FFFFFFF‬;
printf("%d, ", intt);

}

    int c = 0;printf("\n");
}
	__syncthreads();
   for (int s = 1; s <= logN/2; s++) {

	int m = 1 << (2*s);
	int mh = 1 << (2*s - 2);
	int k = m * (Idx / mh);
	int j = Idx%mh;
	float  twiddle_r = cos(-2.0 * j * PI / m);
	float  twiddle_i = sin(-2.0 * j * PI / m);
	
	float  a_r = x_r[k + j];float  a_i = x_i[k + j];
	
	int a = k + j , b = k + j + mh, c = k + j + 2*mh , d = k + j + 3*mh;
   
   
	 
	float  twiddle_factor_r = twiddle_r;
	float  twiddle_factor_i = twiddle_i;
	float  b_r = twiddle_factor_r * x_r[k + j + mh]-twiddle_factor_i * x_i[k + j + mh];
	float  b_i = twiddle_factor_r * x_i[k + j + mh]+twiddle_factor_i * x_r[k + j + mh];
	
	
	float tmp_tw_r =twiddle_factor_r ;float tmp_tw_i = twiddle_factor_i;
	twiddle_factor_r = twiddle_r*tmp_tw_r-twiddle_i*tmp_tw_i;
	twiddle_factor_i = twiddle_r*tmp_tw_i+twiddle_i*tmp_tw_r;
	float  c_r = twiddle_factor_r * x_r[k + j + 2*mh]-twiddle_factor_i * x_i[k + j + 2*mh];
	float  c_i = twiddle_factor_r * x_i[k + j + 2*mh]+twiddle_factor_i * x_r[k + j + 2*mh];
	
	
	tmp_tw_r =twiddle_factor_r ;tmp_tw_i = twiddle_factor_i;
	twiddle_factor_r = twiddle_r*tmp_tw_r-twiddle_i*tmp_tw_i;
	twiddle_factor_i = twiddle_r*tmp_tw_i+twiddle_i*tmp_tw_r;
	float  d_r = twiddle_factor_r * x_r[k + j + 3*mh]-twiddle_factor_i * x_i[k + j + 3*mh];
	float  d_i = twiddle_factor_r * x_i[k + j + 3*mh]+twiddle_factor_i * x_r[k + j + 3*mh];
	// Compute pow(twiddle, j) twiddle_factor *= twiddle;
	
	


	x_r[k + j] = a_r + b_r+c_r+d_r;     	x_i[k + j] = a_i + b_i+c_i+d_i;
	x_r[k + j + mh] = a_r + b_i-c_r-d_i; 	x_i[k + j + mh] = a_i - b_r - c_i + d_r;
	x_r[k + j + 2*mh] = a_r - b_r+c_r - d_r;x_i[k + j + 2*mh] = a_i - b_i + c_i - d_i;
	x_r[k + j + 3*mh] = a_r - b_i-c_r+d_i  ;x_i[k + j + 3*mh] = a_i + b_r - c_i - d_r;
	__syncthreads();
    }


}





// you may define other macros here!
// you may define other functions here!

//-----------------------------------------------------------------------------
void gpuKernel(float* x_r_d, float* x_i_d, /*float* X_r_d, float* X_i_d,*/ const unsigned int N, const unsigned int M)
{
	// In this function, both inputs and outputs are on GPU.
	// No need for cudaMalloc, cudaMemcpy or cudaFree.
	// This function does not run on GPU. 
	// You need to define another function and call it here for GPU execution.
	int R =4; //radix 4
//	dim3 dimGrid = getDimGrid(N/(1024*R)); //modify this function in bmm.cu
//	dim3 dimBlock = getDimBlock(1024); //modify this function in bmm.cu
int cuda_device_ix = 0;
cudaDeviceProp prop;

int size = N>>2;
int block_size = min(size, 1024);
dim3 block(block_size, 1);
dim3 grid((size ) / block_size, 1);
printf("block = %d, grid = %d                %d            %d / %d\n", block_size, (size+1 ) / block_size,N,size,block_size);
	kernelFunc<<< grid,block >>>(x_r_d, x_i_d, N, M); //modify this function in fft.cu


}

 

Overwriting fft.cu


In [28]:
%%writefile fft_main.cu


//Do NOT MODIFY THIS FILE

#include "fft.h"

// ===========================> Functions Prototype <===============================
void fill(float* data, int size);
double calc_mse(float* data1_r, float* data1_i, float* data2_r, float* data2_i, int size);
void get_inputs(int argc, char *argv[], unsigned int& N, unsigned int& M);
void cpuKernel(float* X_serial_r, float* X_serial_i, int n, float* tmp_r, float* tmp_i);
void gpuKernels(float* x_r, float* x_i, float* X_r, float* X_i, unsigned int N, unsigned int M, double* gpu_kernel_time);
// =================================================================================

int main(int argc, char *argv[]) {

    struct cudaDeviceProp p;
    cudaGetDeviceProperties(&p, 0);
    printf("Device Name: %s\n", p.name);

    // get parameters from command line
    unsigned int N, M;
    get_inputs(argc, argv, N, M);

    // allocate memory in CPU for calculation
    float* x_r; // real part
    float* x_i; // imaginary part
    float* X_serial_r;
    float* X_serial_i;
    float* X_r;
    float* X_i;
    x_r = (float*) malloc(N * sizeof(float));
    x_i = (float*) malloc(N * sizeof(float));
    X_serial_r = (float*) malloc(N * sizeof(float));
    X_serial_i = (float*) malloc(N * sizeof(float));
    X_r = (float*) malloc(N * sizeof(float));
    X_i = (float*) malloc(N * sizeof(float));

    // fill x_r and x_i arrays with random values between -8.0f and 8.0f
    srand(0);
   /* fill(x_r, N);
    fill(x_i, N);*/
		for(int pdcnt = 0; pdcnt<N;pdcnt++){
		 x_r[pdcnt] = pdcnt;
		 x_i[pdcnt] = 0;

		}
		
	int i; for (i = 0; i < N; i++) {
		X_serial_r[i] = x_r[i];
		X_serial_i[i] = x_i[i];
	}

    // time measurement for CPU calculation
	float *tmp_r, *tmp_i;
	tmp_r = (float*) malloc(N * sizeof(float));
    tmp_i = (float*) malloc(N * sizeof(float));
    clock_t t0 = clock();
    cpuKernel(X_serial_r, X_serial_i, N, tmp_r, tmp_i);
    clock_t t1 = clock();
	free(tmp_r); free(tmp_i);

    // time measurement for GPU calculation
	double gpu_kernel_time = 0.0;
    clock_t t2 = clock();
	gpuKernels(x_r, x_i, X_r, X_i, N, M, &gpu_kernel_time);
    clock_t t3 = clock();

 	
	/*
	for (i = 0; i<N; i++) {
		printf("%f\t%f\n", X_r[i],X_serial_r[i] );
	}
	printf("\n");
	for (i = 0; i<N; i++) {
		printf("%f\t%f\n", X_i[i] ,X_serial_i[i]);
	}*/
	   // check correctness of calculation
    double mse = calc_mse(X_serial_r, X_serial_i, X_r, X_i, N);
	printf("m=%d n=%d CPU=%g ms GPU=%g ms GPU-Kernels=%g ms mse=%g\n",
	M, N, (t1-t0)/1000.0, (t3-t2)/1000.0, gpu_kernel_time, mse);

	
    // free allocated memory for later use
    free(x_r);
    free(x_i);
    free(X_serial_r);
    free(X_serial_i);
    free(X_r);
    free(X_i);

    return 0;
}

//-----------------------------------------------------------------------------
void gpuKernels(float* x_r, float* x_i, float* X_r, float* X_i, unsigned int N, unsigned int M, double* gpu_kernel_time) {
    float* x_r_d;
    float* x_i_d;
    //float* X_r_d;
    //float* X_i_d;

    HANDLE_ERROR(cudaMalloc((void**)&x_r_d, N * sizeof(float)));
    HANDLE_ERROR(cudaMalloc((void**)&x_i_d, N * sizeof(float)));
    //HANDLE_ERROR(cudaMalloc((void**)&X_r_d, N * sizeof(float)));
    //HANDLE_ERROR(cudaMalloc((void**)&X_i_d, N * sizeof(float)));

    HANDLE_ERROR(cudaMemcpy(x_r_d, x_r, N * sizeof(float), cudaMemcpyHostToDevice));
    HANDLE_ERROR(cudaMemcpy(x_i_d, x_i, N * sizeof(float), cudaMemcpyHostToDevice));

	GpuTimer timer;
    timer.Start();
	gpuKernel(x_r_d, x_i_d, N, M);/*<<<dim3(32,1,1),dim3(32,1,1)>>>(x_r_d, x_i_d, N, M);*/
	timer.Stop();
	*gpu_kernel_time = timer.Elapsed();
	
    //HANDLE_ERROR(cudaMemcpy(X_r, X_r_d, N * sizeof(float), cudaMemcpyDeviceToHost));
    //HANDLE_ERROR(cudaMemcpy(X_i, X_i_d, N * sizeof(float), cudaMemcpyDeviceToHost));
    HANDLE_ERROR(cudaMemcpy(X_r, x_r_d, N * sizeof(float), cudaMemcpyDeviceToHost));
    HANDLE_ERROR(cudaMemcpy(X_i, x_i_d, N * sizeof(float), cudaMemcpyDeviceToHost));

    HANDLE_ERROR(cudaFree(x_r_d));
    HANDLE_ERROR(cudaFree(x_i_d));
    //HANDLE_ERROR(cudaFree(X_r_d));
    //HANDLE_ERROR(cudaFree(X_i_d));
}
//-----------------------------------------------------------------------------
void cpuKernel(float* X_serial_r, float* X_serial_i, int n, float* tmp_r, float* tmp_i) {
	if(n > 1) {	// otherwise, do nothing and return
		int k, m;
		float z_r, z_i, w_r, w_i;
		float *vo_r, *vo_i, *ve_r, *ve_i;
		ve_r = tmp_r; ve_i = tmp_i;
		vo_r = tmp_r + n/2; vo_i = tmp_i + n/2;
		
		for(k=0; k<n/2; k++) {
			ve_r[k] = X_serial_r[2*k]; ve_i[k] = X_serial_i[2*k];
			vo_r[k] = X_serial_r[2*k+1]; vo_i[k] = X_serial_i[2*k+1];
		}
		cpuKernel(ve_r, ve_i, n/2, X_serial_r, X_serial_i);	// FFT on even-indexed elements of v[]
		cpuKernel(vo_r, vo_i, n/2, X_serial_r, X_serial_i);	// FFT on odd-indexed elements of v[]
		
		for(m=0; m<n/2; m++) {
			w_r =  cos((2*PI*m)/n);
			w_i = -sin((2*PI*m)/n);
			z_r = w_r*vo_r[m] - w_i*vo_i[m];	// Re(w*vo[m])
			z_i = w_r*vo_i[m] + w_i*vo_r[m];	// Im(w*vo[m])
			X_serial_r[  m  ] = ve_r[m] + z_r;
			X_serial_i[  m  ] = ve_i[m] + z_i;
			X_serial_r[m+n/2] = ve_r[m] - z_r;
			X_serial_i[m+n/2] = ve_i[m] - z_i;
		}
	}
	return;
}
//-----------------------------------------------------------------------------
void get_inputs(int argc, char *argv[], unsigned int& N, unsigned int& M)
{
    if (
	argc != 2 || 
	atoi(argv[1]) < 0 || atoi(argv[1]) > 26 
	) {
        printf("<< Error >>\n");
        printf("Enter the following command:\n");
        printf("\t./a.out  M\n");
        printf("\t\tM must be between 0 and 26\n");
		exit(-1);
    }
	M = atoi(argv[1]);
    N = (1 << M);
}
//-----------------------------------------------------------------------------
void fill(float* data, int size) {
    for (int i = 0; i < size; i++)
        data[i] = (float)(rand() % 17 - 8);
}
double calc_mse(float* data1_r, float* data1_i, float* data2_r, float* data2_i, int size) {
    double mse = 0.0;
    int i;
    for (i = 0; i < size; i++) {
        double e_r = data1_r[i] - data2_r[i];
				if (e_r <0.5)
					e_r = 0;
				//else
				//	printf("%d %g\t", i, e_r);
        double e_i = data1_i[i] - data2_i[i];
				if (e_i <0.5)
					e_i = 0;
        double e = e_r * e_r + e_i * e_i;
        mse += e;
    }
    return mse/size;
}


Overwriting fft_main.cu


In [29]:
!nvcc -O2 fft_main.cu fft.cu -o fft

fft.cu(84): warning: argument is incompatible with corresponding format string conversion

fft.cu(82): warning: variable "xx" was set but never used

fft.cu(95): warning: variable "c" was declared but never referenced

fft.cu(109): warning: variable "a" was declared but never referenced

fft.cu(109): warning: variable "b" was declared but never referenced

fft.cu(109): warning: variable "c" was declared but never referenced

fft.cu(109): warning: variable "d" was declared but never referenced

fft.cu(160): warning: variable "R" was declared but never referenced

fft.cu(163): warning: variable "cuda_device_ix" was declared but never referenced

fft.cu(164): warning: variable "prop" was declared but never referenced



In [30]:
 !./fft 14

Device Name: 
no CUDA-capable device is detected in fft_main.cu at line 102
